In [30]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from matplotlib import pyplot as plt
import random

### Loading Raw Data

In [3]:
grouped_top_queries = pd.read_csv("./cleaned_data/grouped_top_queries.csv")
grouped_rising_queries = pd.read_csv("./cleaned_data/grouped_rising_queries.csv")

ungrouped_rising_queries = pd.read_csv("C:/Users/varun/exported_files/rising_quer_all.csv")
ungrouped_top_queries = pd.read_csv("C:/Users/varun/exported_files/top_quer_all.csv")

### Visualizing Dataframes

In [15]:
grouped_top_queries.iloc[300:].head()

,TopQueries1,TopQueries2,TopQueries3,TopQueries4,TopQueries5,TopQueries6,TopQueries7,TopQueries8,TopQueries9,TopQueries10,...,TopQueries16,TopQueries17,TopQueries18,TopQueries19,TopQueries20,TopQueries21,TopQueries22,TopQueries23,TopQueries24,TopQueries25
300,feminist,definition of feminism,what is feminism,vatican feminism,intersectional feminism,emma watson feminism,definition of feminism,define feminism,third wave feminism,everyday feminism,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,feminist,definition of feminism,what is feminism,definition of feminism,define feminism,black feminism,definition of feminism,what is feminism,everyday feminism,feminism quotes,...,vatican feminism,what is feminism,equality,feminism tumblr,third wave feminism,clinton feminism,beyonce feminism,anti feminism,feminist,sexism
302,definition of feminism,feminist,definition of feminism,what is feminism,everyday feminism,intersectional feminism,third wave feminism,second wave feminism,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,feminism meaning,feminism meaning,definition of feminism,feminism in india,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,definition of feminism,what is feminism,intersectional feminism,everyday feminism,define feminism,vatican feminism,third wave feminism,definition of feminism,second wave feminism,definition of feminism,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
ungrouped_top_queries.iloc[300:, 2:].head()

,TopQueries1,TopQueries2,TopQueries3,TopQueries4,TopQueries5,TopQueries6,TopQueries7,TopQueries8,TopQueries9,TopQueries10,...,TopQueries16,TopQueries17,TopQueries18,TopQueries19,TopQueries20,TopQueries21,TopQueries22,TopQueries23,TopQueries24,TopQueries25
300,feminist,feminism definition,what is feminism,radical feminism,intersectional feminism,emma watson feminism,feminist definition,define feminism,third wave feminism,everyday feminism,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,feminist,feminism definition,what is feminism,feminist definition,define feminism,black feminism,definition of feminism,white feminism,everyday feminism,feminism quotes,...,radical feminism,what is a feminism,equality,feminism tumblr,first wave feminism,hillary clinton feminism,beyonce feminism,anti feminism,feminists,sexism
302,feminism definition,feminist,definition of feminism,what is feminism,everyday feminism,intersectional feminism,third wave feminism,second wave feminism,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,feminism meaning,feminist meaning,feminism definition,feminism in india,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,feminism definition,what is feminism,intersectional feminism,everyday feminism,define feminism,radical feminism,third wave feminism,feminist definition,second wave feminism,definition of feminism,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Important Functions

In [154]:
import pandas as pd

def compare_dataframes(df1, df2):
    diff_df1_values = []
    diff_df2_values = []

    if df1.shape != df2.shape:
        raise ValueError("Different shapes")

    for col in df1.columns:
        for i in range(df1.shape[0]):
            value1 = df1.loc[i, col]
            value2 = df2.loc[i, col]

            if((str(value1) != str("nan")) & (str(value2) != str("nan"))):
                if value1 != value2:
                    diff_df1_values.append(value1)
                    diff_df2_values.append(value2)

    return diff_df1_values, diff_df2_values


def tool_removal(df, search_terms):
    search_terms_upper = [term.upper() for term in search_terms]

    for col in df.columns:
        for i in range(df.shape[0]):
            cell_value = str(df.loc[i, col]).upper()
            if cell_value in search_terms:
                df.loc[i, col] = np.nan
            elif cell_value in search_terms_upper:
                df.loc[i, col] = np.nan


def random_viewer(nv, ov):
    for i in range(5):
        rand_ind = random.randint(0, len(nv))
        print("New value: " + nv[rand_ind])
        print("Old value: " + ov[rand_ind])
        print("")


def count_uniques(ndf, odf):
    print("New length: " + str(len(np.unique(ndf.values.flatten()))))
    print("Old length: " + str(len(np.unique(odf.values.flatten()))))

### Removing *Search Tools*

In [162]:
tool_list = ["google", "facebook", "twitter", "bing"]
tool_removal(grouped_top_queries, tool_list)

### Storing chnaged values

In [164]:
rising_new_vals, rising_old_vals = compare_dataframes(grouped_rising_queries, ungrouped_rising_queries.iloc[:, 2:])
top_new_vals, top_old_vals = compare_dataframes(grouped_top_queries, ungrouped_top_queries.iloc[:, 2:])

In [165]:
random_viewer(rising_new_vals, rising_old_vals)
print("_________________________________\n")
random_viewer(top_new_vals, top_old_vals)

New value: feminist meaning in tamil
Old value: feminism meaning in telugu

New value: vatican feminism
Old value: radical feminism

New value: feminism meaning
Old value: radical feminism meaning

New value: definition of feminism
Old value: definition of feminist

New value: define feminism
Old value: define feminist

_________________________________

New value: feminism meaning
Old value: pseudo feminism meaning

New value: feminist
Old value: feministing

New value: feminism meaning in malayalam
Old value: feminism meaning in marathi

New value: third wave feminism
Old value: first wave feminism

New value: vatican feminism
Old value: radical feminism



### Counting Unique Values

In [166]:
count_uniques(grouped_rising_queries.applymap(str), ungrouped_rising_queries.iloc[:, 2:].applymap(str))
print("")
count_uniques(grouped_top_queries.applymap(str), ungrouped_top_queries.iloc[:, 2:].applymap(str))

New length: 376
Old length: 581

New length: 259
Old length: 401
